# CQW - Large Reports
Updated: 2019-10-16

Large reports typically return more records. Due to the large number of records returned, total execution time of row versus columnar may be closer. The following reports are found in this notebook. 

  - Report 3a1 - Customer Quantity of Sales 1999-2002 where Lifetime Quantity > 16,000
  - Report 3a2 - Customer Quantity of Sales 1999-2002
  - Report 3b1 - Sorted List Customers Quantity Sold for Year 1999

### Load Db2 Extensions and Connect to the Database
The `connection` notebook contains the `CONNECT` statement which allows access to the `SAMPLE` database. If you need to modify the connection information, edit the `connection.ipynb` notebook.

In [ ]:
%run ../db2.ipynb
%run ../connection.ipynb

### Create a Function for Running the Queries
This function is used to time the queries and return the total time taken. The parameter is the SQL that will be run and the function will execute against the row-based and col-based tables. 

In [ ]:
def runquery(sql):
    
    %sql -q SET CURRENT SCHEMA = CQWROW
        
    print("Starting execution of ROW query")    
    start_time = time.time()

    %sql -q {sql}
    
    end_time = time.time()
    row = end_time - start_time
    
    print("Row run time = {0}".format(row))
    
    %sql -q SET CURRENT SCHEMA = CQWCOL
    
    print()
    print("Starting execution of COLUMNAR query")        
    
    start_time = time.time()

    results = %sql -q {query}
    
    end_time = time.time()
    col = end_time - start_time
    
    print("Column run time = {0}".format(col))    
    
    pdisplay(results)
    
    %sql -bar VALUES ('Row',:row),('Column',:col)

## Customer Quantity of Sales 1999-2002 where Lifetime Quantity > 16,000

In [ ]:
query = '''
WITH 
"RSF_Query1" AS 
    (
    SELECT
        "CUSTOMER"."C_CUSTOMER_SK" AS "Customers_C_CUSTOMER_SK", 
        SUM(CAST("STORE_SALES"."SS_QUANTITY" AS DECIMAL(31,2))) AS "Quantity__Store_Sales_"
    FROM
        "CUSTOMER" "CUSTOMER"
            INNER JOIN "STORE_SALES" "STORE_SALES"
            ON "CUSTOMER"."C_CUSTOMER_SK" = "STORE_SALES"."SS_CUSTOMER_SK" 
    GROUP BY 
        "CUSTOMER"."C_CUSTOMER_SK"
    )
SELECT
    "RSF_Query1"."Customers_C_CUSTOMER_SK" AS "Customers_C_CUSTOMER_SK", 
    "RSF_Query1"."Quantity__Store_Sales_" AS "Quantity__Store_Sales_"
FROM
    "RSF_Query1" 
WHERE 
    "RSF_Query1"."Quantity__Store_Sales_" > 16000.0
'''
runquery(query)

## Customer Quantity of Sales 1999-2002 

In [ ]:
query = '''
SELECT
    "DATE_DIM"."D_YEAR" AS "Date_D_YEAR", 
    "CUSTOMER"."C_CUSTOMER_SK" AS "Customers_C_CUSTOMER_SK", 
    SUM(CAST("STORE_SALES"."SS_QUANTITY" AS DECIMAL(31,2))) AS "Quantity__Store_Sales_"
FROM
    "CUSTOMER" "CUSTOMER"
        INNER JOIN "STORE_SALES" "STORE_SALES"
        ON "CUSTOMER"."C_CUSTOMER_SK" = "STORE_SALES"."SS_CUSTOMER_SK"
            INNER JOIN "DATE_DIM" "DATE_DIM"
            ON "DATE_DIM"."D_DATE_SK" = "STORE_SALES"."SS_SOLD_DATE_SK" 
WHERE 
    "CUSTOMER"."C_CUSTOMER_SK" BETWEEN -99999 AND 11999979 
GROUP BY 
    "DATE_DIM"."D_YEAR", 
    "CUSTOMER"."C_CUSTOMER_SK"
'''
runquery(query)

## Sorted List Customers Quantity Sold for Year 1999

In [ ]:
query = '''
WITH 
"SQ0_Query1" AS 
    (
    SELECT
        "DATE_DIM"."D_YEAR" AS "Date_D_YEAR", 
        "CUSTOMER"."C_CUSTOMER_SK" AS "Customers_C_CUSTOMER_SK", 
        SUM(CAST("STORE_SALES"."SS_QUANTITY" AS DECIMAL(31,2))) AS "Quantity__Store_Sales_"
    FROM
        "CUSTOMER" "CUSTOMER"
            INNER JOIN "STORE_SALES" "STORE_SALES"
            ON "CUSTOMER"."C_CUSTOMER_SK" = "STORE_SALES"."SS_CUSTOMER_SK"
                INNER JOIN "DATE_DIM" "DATE_DIM"
                ON "DATE_DIM"."D_DATE_SK" = "STORE_SALES"."SS_SOLD_DATE_SK" 
    WHERE 
        "DATE_DIM"."D_YEAR" > 1899 AND "DATE_DIM"."D_YEAR" < 2000
    GROUP BY 
        "DATE_DIM"."D_YEAR", 
        "CUSTOMER"."C_CUSTOMER_SK"
    )
SELECT
    "SQ0_Query1"."Date_D_YEAR" AS "Date_D_YEAR", 
    "SQ0_Query1"."Customers_C_CUSTOMER_SK" AS "Customers_C_CUSTOMER_SK", 
    "SQ0_Query1"."Quantity__Store_Sales_" AS "Quantity__Store_Sales_", 
    RANK()
        OVER(
            ORDER BY
                "SQ0_Query1"."Quantity__Store_Sales_" DESC NULLS LAST
        ) AS "rank0"
FROM
    "SQ0_Query1"
'''
runquery(query)

#### Credits: IBM 2019, George Baklarz [baklarz@ca.ibm.com]